In [1]:
# Run order - 2
# Needed input files: 'chemistries_git.csv', 'second_genome_2.csv', 'data_discovery.csv', 'top_11_mets.csv'
# Generated output files: None

In [2]:
# Load libraries
import sklearn
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn import model_selection
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import cross_val_score
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
import random
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [3]:
#Importing discovery cohort Clinical labs data
chemistries_discovery=pd.read_csv('chemistries_git.csv', 
                          index_col = 'public_client_id')
chemistries_discovery.index=chemistries_discovery.index.astype('float64')
#check correct sample size
print (chemistries_discovery.shape)
#Importing Validation cohort metabolomics data
second_genome=pd.read_csv('second_genome_2.csv', 
                          index_col = 'public_client_id')
second_genome.index=second_genome.index.astype('float64')
#check correct sample size
print (second_genome.shape)
#Importing discovery metabolomics data
discovery_mets=pd.read_csv('data_discovery.csv', 
                          index_col = 'public_client_id')
discovery_mets.index=discovery_mets.index.astype('float64')
#check correct sample size
print (discovery_mets.shape)
#import list of top 11 metabolites from LASSSO
top_11_=pd.read_csv('top_11_mets.csv', 
                          index_col = 0)
top_11_=top_11_.index.tolist()

(399, 78)
(540, 660)
(399, 665)


In [4]:
#Scale and standardize metabolites discovery
X = discovery_mets[discovery_mets.columns[0:659]]
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
Xcolumns=X.columns
X = scaler.fit_transform(X)
X=pd.DataFrame(data=X,columns=Xcolumns)
print (X.shape)

(399, 659)


In [5]:
#Scale and standardize metabolites validation
vendor = second_genome[second_genome.columns[0:659]]
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
Xcolumns=vendor.columns
vendor = scaler.fit_transform(vendor)
vendor=pd.DataFrame(data=vendor,columns=Xcolumns)
print (vendor.shape)

(540, 659)


In [6]:
#Prepare data for RF classifier
classifier=pd.DataFrame(index=discovery_mets.index,data=X.set_index(discovery_mets.index))
classifier_validation=pd.DataFrame(index=second_genome.index,data=vendor.set_index(second_genome.index))
#Retain only the 11 metabolites in df
for x in classifier.columns.tolist():
    if x not in top_11_:
        classifier.drop([x],1,inplace=True)
print(classifier.shape)
for x in classifier_validation.columns.tolist():
    if x not in top_11_:
        classifier_validation.drop([x],1,inplace=True)
print (classifier_validation.shape)
#Generate quartiles
classifier['shannon_quant']=pd.qcut(discovery_mets['shannon'],4,labels=False)
classifier_validation['shannon_quant']=pd.qcut(second_genome['shannon'],4,labels=False)
#recode quartiles
a=[]
for x in classifier['shannon_quant']:
    if x==0:
        a.append(1)
    else:
        a.append(0)
classifier['shannon_quant']=a
print (classifier['shannon_quant'].sum())
classifier.drop(['shannon_quant'],1,inplace=True)
b=[]
for x in classifier_validation['shannon_quant']:
    if x==0:
        b.append(1)
    else:
        b.append(0)
classifier_validation['shannon_quant']=b
print (classifier_validation['shannon_quant'].sum())
classifier_validation.drop(['shannon_quant'],1,inplace=True)
shannon_clf = RandomForestClassifier(n_estimators=2000,
                                 criterion='entropy',
                                 bootstrap=True, oob_score=False,class_weight={0:1,1:3.5},
                                 max_features='sqrt',
                                 min_samples_split=2,
                                 min_samples_leaf=2,
                                 max_depth=5,
                                 random_state=1,
                                 n_jobs=1)

(399, 11)
(540, 11)
100
135


# Running Random forest 10-fold CV on the discovery cohort

In [7]:
X = classifier.reset_index()
X.drop(['public_client_id'],1,inplace=True)
X=np.array(X)
y=np.array(a)
n_samples, n_features = X.shape


# Perform 10-fold RF CV, extract sensitivity, specificity, precision and AUCs for each test set
FOLDS=10
cv = StratifiedKFold(n_splits=10)
reversed_mean_precision = 0.0
mean_recall = np.linspace(0, 1, 100)
mean_precision=[]
pr_auc=[]
tprs = []
aucs = []
total_specificity=[]
total_sensitivity=[]
mean_fpr = np.linspace(0, 1, 100)
i = 0
for train, test in cv.split(X, y):
    probas_ = shannon_clf.fit(X[train], y[train]).predict_proba(X[test])
    preds=shannon_clf.predict(X[test])
    tn, fp, fn, tp = confusion_matrix(y[test],preds).ravel()
    sensitivity=(tp/(tp+fn))
    specificity=(tn/(tn+fp))
    mean_precision.append(precision_score(y[test],preds))
    total_specificity.append(specificity)
    total_sensitivity.append(sensitivity)
    precision, recall, thresholds= precision_recall_curve(y[test], probas_[:, 1])
    lab = 'Fold %d PR AUC=%.4f' % (i+1, auc(recall, precision))
    pr_auc.append(auc(recall, precision))
    print (lab)
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    reversed_recall = np.fliplr([recall])[0]
    reversed_precision = np.fliplr([precision])[0]
    reversed_mean_precision += interp(mean_recall, reversed_recall, reversed_precision)
    reversed_mean_precision[0] = 0.0
    tprs[-1][0]=0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    i += 1
reversed_mean_precision /= FOLDS
reversed_mean_precision[0] = 1
mean_tpr_discovery = np.mean(tprs, axis=0)
mean_tpr_discovery[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr_discovery)
print ('mean sensitivity discovery',np.mean(total_sensitivity))
print ('mean specificity discovery',np.mean(total_specificity))
print ('mean precision discovery',np.mean(mean_precision))
print ('mean PR AUC',np.mean(pr_auc))
print ('std dev.',np.std(pr_auc))
print ('mean ROC AUC',np.mean(aucs))
print ('std_dev ROC AUC',np.std(aucs))
mean_AUC_disc=np.mean(pr_auc)
ROC_AUC_disc=np.mean(aucs)

Fold 1 PR AUC=0.8054
Fold 2 PR AUC=0.5833
Fold 3 PR AUC=0.8264
Fold 4 PR AUC=0.6564
Fold 5 PR AUC=0.8055
Fold 6 PR AUC=0.6580
Fold 7 PR AUC=0.8190
Fold 8 PR AUC=0.8111
Fold 9 PR AUC=0.8450
Fold 10 PR AUC=0.7892
mean sensitivity discovery 0.72
mean specificity discovery 0.9031034482758621
mean precision discovery 0.7206313131313131
mean PR AUC 0.7599295308547271
std dev. 0.08665212426781399
mean ROC AUC 0.8840689655172413
std_dev ROC AUC 0.044473556326354025


# Running Random forest 10-fold CV on the validation cohort

In [8]:
# Perform 10-fold CV RF classification on the validation set, extract performance metrics for each out-of-sample set
X = classifier_validation.reset_index()
X.drop(['public_client_id'],1,inplace=True)
X=np.array(X)
y=np.array(b)
n_samples, n_features = X.shape

FOLDS=10
cv = StratifiedKFold(n_splits=10)
val_reversed_mean_precision = 0.0
val_mean_recall = np.linspace(0, 1, 100)
mean_precision=[]
pr_auc=[]
tprs = []
aucs = []
total_specificity=[]
total_sensitivity=[]
mean_fpr = np.linspace(0, 1, 100)
i = 0
for train, test in cv.split(X, y):
    probas_ = shannon_clf.fit(X[train], y[train]).predict_proba(X[test])
    preds=shannon_clf.predict(X[test])
    tn, fp, fn, tp = confusion_matrix(y[test],preds).ravel()
    sensitivity=(tp/(tp+fn))
    specificity=(tn/(tn+fp))
    mean_precision.append(precision_score(y[test],preds))
    total_specificity.append(specificity)
    total_sensitivity.append(sensitivity)
    precision, recall, thresholds= precision_recall_curve(y[test], probas_[:, 1])
    lab = 'Fold %d PR AUC=%.4f' % (i+1, auc(recall, precision))
    pr_auc.append(auc(recall, precision))
    print (lab)
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    reversed_recall = np.fliplr([recall])[0]
    reversed_precision = np.fliplr([precision])[0]
    val_reversed_mean_precision += interp(val_mean_recall, reversed_recall, reversed_precision)
    val_reversed_mean_precision[0] = 0.0
    tprs[-1][0]=0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    i += 1
val_reversed_mean_precision /= FOLDS
val_reversed_mean_precision[0] = 1
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
std_auc = np.std(aucs)
print ('Validation mean sensitivity',np.mean(total_sensitivity))
print ('Validation mean specificity',np.mean(total_specificity))
print ('Validation mean precision',np.mean(mean_precision))
print ('mean PR AUC',np.mean(pr_auc))
print ('std_dev PR AUC',np.std(pr_auc))
print ('mean ROC AUC',np.mean(aucs))
print ('std_dev ROC AUC',np.std(aucs))

Fold 1 PR AUC=0.6646
Fold 2 PR AUC=0.6953
Fold 3 PR AUC=0.5521
Fold 4 PR AUC=0.8411
Fold 5 PR AUC=0.4499
Fold 6 PR AUC=0.8148
Fold 7 PR AUC=0.6656
Fold 8 PR AUC=0.6835
Fold 9 PR AUC=0.5826
Fold 10 PR AUC=0.7024
Validation mean sensitivity 0.651098901098901
Validation mean specificity 0.869329268292683
Validation mean precision 0.643225238813474
mean PR AUC 0.6651764096452646
std_dev PR AUC 0.11041580011801376
mean ROC AUC 0.81774457250067
std_dev ROC AUC 0.06996473620780753


# Assessing Predictive Capacity of Clinical Labs

In [9]:
#Assessing classification capacity of clinical labs
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
C = chemistries_discovery.loc[:,chemistries_discovery.columns!='shannon']
y = chemistries_discovery['shannon']
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
Ccolumns=C.columns
C = scaler.fit_transform(C)
C=pd.DataFrame(data=C,columns=Ccolumns)
print (C.shape)

(399, 77)


In [10]:
#Classifying participants in bottom quartile of shannon using 77 clinical labs
classifier_chem=pd.DataFrame(index=discovery_mets.index,data=C.set_index(discovery_mets.index))
classifier_chem['shannon_quant']=pd.qcut(chemistries_discovery['shannon'],4,labels=False)
c=[]
for x in classifier_chem['shannon_quant']:
    if x==0:
        c.append(1)
    else:
        c.append(0)
classifier_chem['shannon_quant']=c
print (classifier_chem['shannon_quant'].sum())
classifier_chem.drop(['shannon_quant'],1,inplace=True)
print (classifier_chem.shape)

100
(399, 77)


In [11]:
shannon_clf = RandomForestClassifier(n_estimators=2000,
                                 criterion='entropy',
                                 bootstrap=False, oob_score=False,class_weight={0:1,1:3.5},
                                 max_features='auto',
                                 min_samples_split=2,
                                 min_samples_leaf=5,
                                 max_depth=5,
                                 random_state=1,
                                 n_jobs=1)

In [12]:
#using cross_val_Score to get mean ROC_auc, and comparing it to scores extracted independently for each CV
chem_score=cross_val_score(shannon_clf,classifier_chem, c,cv=10,scoring='roc_auc')
print('mean auc auc clinical labs discovery',np.mean(chem_score))
print('std deviation auc chemistries discovery',np.std(chem_score))

mean auc auc clinical labs discovery 0.6675632183908046
std deviation auc chemistries discovery 0.05156583181943268


In [13]:
X = classifier_chem.reset_index()
X.drop(['public_client_id'],1,inplace=True)
X=np.array(X)
y=np.array(c)
n_samples, n_features = X.shape

cv = StratifiedKFold(n_splits=10)
clinical_reversed_mean_precision = 0.0
clinical_mean_recall = np.linspace(0, 1, 100)
features=pd.DataFrame(index=C.columns)
mean_precision=[]
pr_auc_clinical=[]
tprs = []
aucs = []
total_specificity=[]
total_sensitivity=[]
mean_fpr = np.linspace(0, 1, 100)

i = 0
for train, test in cv.split(X, y):
    probas_ = shannon_clf.fit(X[train], y[train]).predict_proba(X[test])
    features[i]=list(shannon_clf.feature_importances_)
    preds=shannon_clf.predict(X[test])
    tn, fp, fn, tp = confusion_matrix(y[test],preds).ravel()
    sensitivity=(tp/(tp+fn))
    specificity=(tn/(tn+fp))
    mean_precision.append(precision_score(y[test],preds))
    total_specificity.append(specificity)
    total_sensitivity.append(sensitivity)
    precision, recall, thresholds= precision_recall_curve(y[test], probas_[:, 1])
    lab = 'Fold %d AUC=%.4f' % (i+1, auc(recall, precision))
    pr_auc_clinical.append(auc(recall, precision))
    print (lab)
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    reversed_recall = np.fliplr([recall])[0]
    reversed_precision = np.fliplr([precision])[0]
    clinical_reversed_mean_precision += interp(mean_recall, reversed_recall, reversed_precision)
    clinical_reversed_mean_precision[0] = 0.0
    tprs[-1][0]=0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    i += 1
clinical_reversed_mean_precision/= FOLDS
clinical_reversed_mean_precision[0] = 1
mean_tpr_chemistries = np.mean(tprs, axis=0)
mean_tpr_chemistries[-1] = 1.0
print ('precision',np.mean(mean_precision))
print ('specificity',np.mean(total_specificity))
print ('sensitivity',np.mean(total_sensitivity))
print ('pr_AUC',np.mean(pr_auc_clinical))
print ('pr_AUC std. dev.',np.std(pr_auc_clinical))
print ('ROC_AUC',np.mean(np.mean(aucs)))
print ('ROC_AUC std. dev.',np.std(aucs))
mean_ROC_clinical=np.mean(aucs)

Fold 1 AUC=0.5674
Fold 2 AUC=0.3950
Fold 3 AUC=0.3864
Fold 4 AUC=0.5904
Fold 5 AUC=0.3439
Fold 6 AUC=0.6221
Fold 7 AUC=0.6207
Fold 8 AUC=0.4133
Fold 9 AUC=0.2947
Fold 10 AUC=0.5367
precision 0.4536671661671662
specificity 0.7960919540229885
sensitivity 0.5
pr_AUC 0.4770714585758976
pr_AUC std. dev. 0.11670937681743301
ROC_AUC 0.6675632183908046
ROC_AUC std. dev. 0.05156583181943268
